# <center>ESG Mispricing: A View from the Overnight Return</center>

<p></p> 
<center><b>Xiaoqun Liu</b> </center> 
<p></p>

<center><b>Changrong Yang</b> </center> 
<p></p> 
<center><b>Junjie Shi</b></center> 
<p></p>

<center>Department of Finance, Hainan University</center>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import datetime
import os
from regression_demo import *
from regression_demo import __fama_macbeth_coef__,_format_reg_
from portfolios1D import *
from portfolios2D import *

# environment
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_row', None)
NWlags = [0,4,4]

In [2]:
import IPython.core.display as di

# this line will add a button to toggle visibility of code blocks
di.display_html('''<button onclick="jQuery('.input_area').toggle(); 
                   jQuery('.prompt').toggle();">Toggle code</button>''', 
                raw=True)

# this line will output variables without using print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

Toggle code

In [3]:
year_data_origin = pd.read_csv(r'../Data/year_indicator.csv',dtype={"Stkcd":str,'Trdmn':str,'Trdyear':str})
year_data_origin = year_data_origin.sort_values(['Stkcd','Trdyear'],ascending=[1,1]).reset_index(drop=True)
year_data_origin = year_data_origin.dropna().reset_index(drop=True)
year_data_origin = year_data_origin.query("Trdyear!='2020'").reset_index(drop=True)
for i in range(2, len(year_data_origin.columns)):
    year_data_origin[year_data_origin.columns[i]] = winsorize(year_data_origin[year_data_origin.columns[i]], (0.01, 0.01))

In [4]:
year_ff5 = pd.read_csv(r'../Data/fivefactor_yearly.csv',dtype={'trdyr':str}, low_memory=False, sep=',')
year_ff5 = year_ff5.rename(columns={'trdyr': 'Trdyear'})
year_ff5 = year_ff5.sort_values(["Trdyear"], ascending=[1]).reset_index(drop=True)
year_ff5 = year_ff5.dropna().reset_index(drop=True)
year_ff5 = year_ff5[['Trdyear','mkt_rf','smb','hml','umd','rmw','cma','rf']].copy()

# Table 1

In [5]:
des = year_data_origin[['ESG_score','E_score','S_score','G_score','year_return','overnight_return','intraday_return','size','BM','ILLIQ','turnover']].describe(percentiles=[.05,.25,.5,.75,.95])
des = des.loc[['count','mean','std','5%','25%','50%','75%','95%']]
des.columns = [
    "$ESG_{score}$",
    '$E_{score}$',
    '$S_{score}$',
    '$G_{score}$',
    '$R_{close-to-close}$',
    '$R_{overnight}$',
    '$R_{intraday}$',
    '$Size$',
    '$BM$',
    '$Illiquidity$',
    '$Turnover$'
]
des.index = ['Observation','Mean','STD','5%','25%','Median','75%','95%']
des = des.round(decimals=3)
des.loc["Observation"] = ["{0:,d}".format(int(i)) for i in des.loc["Observation"]]
des.T

,Observation,Mean,STD,5%,25%,Median,75%,95%
$ESG_{score}$,"3,096",22.385,6.057,15.289,19.008,21.074,23.967,36.58
$E_{score}$,"3,096",11.537,7.315,2.326,6.977,10.078,13.953,28.682
$S_{score}$,"3,096",25.977,8.619,12.281,22.807,22.807,28.07,43.86
$G_{score}$,"3,096",43.852,5.221,33.929,39.286,42.857,48.214,53.571
$R_{close-to-close}$,"3,096",0.082,0.436,-0.457,-0.228,-0.013,0.301,0.944
$R_{overnight}$,"3,096",-0.191,0.225,-0.532,-0.338,-0.204,-0.071,0.193
$R_{intraday}$,"3,096",0.417,0.697,-0.289,-0.032,0.219,0.651,1.789
$Size$,"3,096",16.049,1.099,14.324,15.298,15.972,16.705,17.917
$BM$,"3,096",0.696,0.256,0.242,0.503,0.715,0.908,1.071
$Illiquidity$,"3,096",0.035,0.04,0.003,0.01,0.022,0.043,0.114


# Table 2

In [6]:
year_data = year_data_origin.copy()
year_data['ESG_score_lag1'] = year_data.groupby(['Stkcd'])['ESG_score'].shift(1).reset_index(drop=True)
year_data['size_lag1'] = year_data.groupby('Stkcd')['size'].shift(1).reset_index(drop=True)
year_data =year_data.dropna().reset_index(drop=True)

In [7]:
formulas = [
    'weight_r~1',
    'weight_r~1+mkt_rf+smb+hml',
    'weight_r~1+mkt_rf+smb+hml+rmw+cma']

In [8]:
groups = ['ESG_score']
all_alphas=[]
for group in groups:
    year_data[group+'_decile']= 1 + year_data.groupby('Trdyear')[group+'_lag1'].transform(lambda x: pd.qcut(x, 5, labels=False, duplicates="drop"))
    portfolio_return=form_portfolio_1D(year_data,year_ff5,[group],['Trdyear'],'overnight_return','size_lag1')
    portfolio_data=pd.merge(portfolio_return,year_ff5,on=['Trdyear'],how='inner')
    for NWlag,formula in zip(NWlags,formulas):
        alphas = portfolio_data.groupby(group+'_decile').apply(alpha,formula, NWlag).reset_index(level=0).rename(columns={'Coef.': 'alpha'})
        alphas.index = ['$Low$ $ESG_{score}$','2','3','4','$High$ $ESG_{score}$','$High$-$Low$']
        alphas = alphas.drop(['ESG_score_decile','Num.Obs.','R-squared'],axis=1)
        alphas = alphas.apply({
                'alpha':str,
                't-stat.': '{:,.2f}'.format,
                })
        t_value = alphas[['t-stat.']].loc[['$High$-$Low$']].apply(lambda x:'('+x+')').rename(columns={'t-stat.':'alpha'})
        alphas = alphas[['alpha']].append(t_value)
        all_alphas.append(alphas)
alphas=all_alphas[0] 
alphas.columns=['Average excess return']
alphas["Three-factor (FF) alpha"]=all_alphas[1]['alpha']
alphas["Five-factor (FF) alphaa"]=all_alphas[2]['alpha']
alphas.index = ['$Low$ $ESG_{score}$','2','3','4','$High$ $ESG_{score}$','$High$-$Low$',""]

alphas


,Average excess return,Three-factor (FF) alpha,Five-factor (FF) alphaa
$Low$ $ESG_{score}$,-0.2361***,-0.2466***,-0.2192***
2,-0.2116***,-0.2211***,-0.1576***
3,-0.1871***,-0.1966***,-0.1574***
4,-0.2288***,-0.2383***,-0.1763***
$High$ $ESG_{score}$,-0.1984***,-0.2089***,-0.1523***
$High$-$Low$,0.0377***,0.0377***,0.0669***
,(3.58),(8.41),(5.70)


# Table 3

In [9]:
year_data = year_data_origin.copy()
year_data['year_return_rf_lead1'] = year_data.groupby('Stkcd')['year_return_rf'].shift(-1).reset_index(drop=True)
year_data['overnight_return_rf_lead1'] = year_data.groupby('Stkcd')['overnight_return_rf'].shift(-1).reset_index(drop=True)
year_data['intraday_return_rf_lead1'] = year_data.groupby('Stkcd')['intraday_return_rf'].shift(-1).reset_index(drop=True)
year_data = year_data.dropna().reset_index(drop=True)

In [10]:
formulas = [
    'overnight_return_rf_lead1 ~ 1 + ESG_score + turnover + size + BM + ILLIQ',
    'overnight_return_rf_lead1 ~ 1 + E_score + turnover + size + BM + ILLIQ',
    'overnight_return_rf_lead1 ~ 1 + S_score + turnover + size + BM + ILLIQ',
    'overnight_return_rf_lead1 ~ 1 + G_score + turnover + size + BM + ILLIQ',
]

In [11]:
regressor_order = ['ESG_score', 'E_score','S_score','G_score','size','BM','turnover','ILLIQ','Intercept']
info_list = ['No. Observations','AdjR2_mean']
format_results = []

for i, formula in enumerate(formulas):
    model_name = "M" + str(i + 1)
    ols_result  = year_data.groupby('Trdyear').apply(__fama_macbeth_coef__, formula, NWlag=1).reset_index(level=0,drop=True)
    fama_result = fama_macbeth(year_data, formula, ols_result, NWlag=1, model_name=model_name)
    format_results.append(fama_result)
results = merge_results(format_results, regressor_order, info_list)
results.loc["No. Observations"] = ["{0:,d}".format(int(i)) for i in results.loc["No. Observations"]]
results.index = ['$ESG_{score}$','','$E_{score}$','','$S_{score}$','','$G_{score}$','',
                 '$Size$','','$BM$','','$Turnover$','','$Illiquidity$','','$Intercept$','','$Observations$','$R^2$']
results

,M1,M2,M3,M4
$ESG_{score}$,-0.003***,,,
,(-4.43),,,
$E_{score}$,,-0.003***,,
,,(-3.03),,
$S_{score}$,,,-0.001***,
,,,(-5.59),
$G_{score}$,,,,0.000
,,,,(0.12)
$Size$,0.023***,0.023***,0.020***,0.016***
,(4.15),(4.04),(3.85),(3.38)


# Table 4

In [12]:
year_data = year_data_origin.copy()
year_data['ESG_score_lag1'] = year_data.groupby(['Stkcd'])['ESG_score'].shift(1).reset_index(drop=True)
year_data['size_lag1'] = year_data.groupby('Stkcd')['size'].shift(1).reset_index(drop=True)
year_data['BM_lag1'] = year_data.groupby('Stkcd')['BM'].shift(1).reset_index(drop=True)
year_data['ILLIQ_lag1'] = year_data.groupby('Stkcd')['ILLIQ'].shift(1).reset_index(drop=True)
year_data =year_data.dropna().reset_index(drop=True)

In [13]:
formulas = ['weight_r~1+mkt_rf+smb+hml+rmw+cma']

In [14]:
groups = ['size','BM','ILLIQ']
# titles=["Panel A: Double-sorted portfolios based on Size and ESGscore","Panel B: Double-sorted portfolios based on BM and ESGscore","Panel C: Double-sorted portfolios based on Illiquidity and ESGscore"]
for group in groups:
    year_data[group + '_decile']= 1 + year_data.groupby('Trdyear')[group+ '_lag1'].transform(lambda x: pd.qcut(x, 5, labels=False, duplicates="drop"))
    year_data['ESG_score_decile']= 1 + year_data.groupby(['Trdyear',group + '_decile'])['ESG_score_lag1'].transform(lambda x: pd.qcut(x, 5, labels=False, duplicates="drop"))
    portfolio_return=form_portfolio_2D_1(year_data,year_ff5,[group,'ESG_score'],['Trdyear'],'overnight_return','size_lag1')
    portfolio_data=pd.merge(portfolio_return,year_ff5,on=['Trdyear'],how='inner')
    formula='weight_r~1+mkt_rf+smb+hml+rmw+cma'
    alphas = portfolio_data.groupby(['ESG_score_decile',group + '_decile']).apply(alpha,formula, NWlag=1).rename(columns={'Coef.': 'alpha'})
    alphas = alphas.apply({
            'alpha':str,
            't-stat.': '{:,.2f}'.format,
            'Num.Obs.': '{:,.0f}'.format,
            'R-squared': '{:,.2%}'.format
            }).reset_index()

    alphas['$ESG_{score}$ $decile$'] = np.where(alphas['ESG_score_decile']==1,'$Low$ $ESG_{score}$',alphas['ESG_score_decile'])
    alphas['$ESG_{score}$ $decile$'] = np.where(alphas['$ESG_{score}$ $decile$']==5,'$High$ $ESG_{score}$',alphas['$ESG_{score}$ $decile$'])
    alphas['$ESG_{score}$ $decile$'] = np.where(alphas['$ESG_{score}$ $decile$']=='top-bottom_ESG_score_decile','$High$-$Low$',alphas['$ESG_{score}$ $decile$'])
    alphas[group+"$decile$"] = np.where(alphas[group + '_decile']==1,'$Low$'+group,alphas[group + '_decile'])
    alphas[group+"$decile$"] = np.where(alphas[group+"$decile$"]==5,'$High$'+group,alphas[group+"$decile$"])
    alphas[group+"$decile$"] = np.where(alphas[group+"$decile$"]=='top-bottom_'+group +'_decile','$High$-$Low$',alphas[group+"$decile$"])
    alphas['$ESG_{score}$ $decile$'] = alphas['$ESG_{score}$ $decile$'].astype(str)
    alphas[group+"$decile$"] = alphas[group+"$decile$"].astype(str)
    alphas = alphas[(alphas['$ESG_{score}$ $decile$']!='2')&(alphas['$ESG_{score}$ $decile$']!='3')&(alphas['$ESG_{score}$ $decile$']!='4')]
    alphas = alphas.drop(['ESG_score_decile',group +'_decile','Num.Obs.','R-squared'],axis=1).rename(columns={'level_2':' '})
    alphas = alphas.set_index(['$ESG_{score}$ $decile$',group+'$decile$',' '])
    alphas = alphas.loc[['$Low$ $ESG_{score}$','$High$ $ESG_{score}$','$High$-$Low$']]
    alphas = alphas.unstack(1).loc[['$Low$ $ESG_{score}$','$High$ $ESG_{score}$','$High$-$Low$']]
    t_value = alphas['t-stat.'][['$Low$'+group,'2','3','4','$High$'+group]].loc[['$High$-$Low$']].apply(lambda x:'('+x+')')
    alphas = alphas['alpha'][['$Low$'+group,'2','3','4','$High$'+group,'$High$-$Low$']]
    alphas = alphas[['$Low$'+group,'2','3','4','$High$'+group]].append(t_value)

    alphas

size$decile$               $Low$size           2           3           4  $High$size
$ESG_{score}$ $decile$                                                              
$Low$ $ESG_{score}$    0  -0.2668***  -0.2819***  -0.1765***  -0.1379***  -0.0732***
$High$ $ESG_{score}$   0  -0.2535***  -0.2055***  -0.1472***  -0.1589***   -0.1428**
$High$-$Low$           0     -0.0073    0.0559**      0.0088     -0.0416   -0.0902**
                       0     (-0.79)      (2.50)      (1.17)     (-1.45)     (-2.38)

BM$decile$                   $Low$BM           2           3           4    $High$BM
$ESG_{score}$ $decile$                                                              
$Low$ $ESG_{score}$    0  -0.2597***  -0.2378***  -0.2181***  -0.2094***  -0.1101***
$High$ $ESG_{score}$   0  -0.2323***  -0.1907***  -0.1363***  -0.1289***  -0.1253***
$High$-$Low$           0      0.0068      0.0266     0.0612*     0.06***    -0.0357*
                       0      (0.64)      (0.85)      (1.86)      (6.60)     (-1.95)

ILLIQ$decile$             $Low$ILLIQ           2           3           4 $High$ILLIQ
$ESG_{score}$ $decile$                                                              
$Low$ $ESG_{score}$    0  -0.1214***  -0.1883***  -0.2084***  -0.1785***  -0.2594***
$High$ $ESG_{score}$   0   -0.1329**  -0.1924***  -0.1301***  -0.1818***  -0.2239***
$High$-$Low$           0    -0.032**  -0.0247***   0.0577***    -0.0238*       0.015
                       0     (-2.37)     (-2.90)      (3.21)     (-1.75)      (0.47)